In [23]:
from grimsel.core.io import IO
import numpy as np
import pandas as pd
import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB
from grimsel import logger
logger.setLevel(0)

In [54]:
df_def_node = pd.DataFrame({'nd': ['Node1'], 'price_co2': [40], 'nd_weight': [1]})
df_def_node.index.name = 'nd_id'
df_def_node = df_def_node.reset_index()
dict_nd = df_def_node.set_index('nd').nd_id.to_dict()
display(df_def_node)

,nd_id,nd,price_co2,nd_weight
0,0,Node1,40,1


In [55]:
df_def_fuel = pd.DataFrame({'fl': ['natural_gas', 'hard_coal', 'photovoltaics'], 
                            'co2_int': [0.20196, 0.34596, 0], 
                            })
df_def_fuel.index.name = 'fl_id'
df_def_fuel = df_def_fuel.reset_index()
dict_fl = df_def_fuel.set_index('fl').fl_id.to_dict()
display(df_def_fuel)

,fl_id,fl,co2_int
0,0,natural_gas,0.20196
1,1,hard_coal,0.34596
2,2,photovoltaics,0.00000


In [56]:
df_def_encar = pd.DataFrame({'ca_id': [0],
                             'ca': ['EL']})
df_def_encar.index.name = 'ca_id'
dict_ca = df_def_encar.set_index('ca').ca_id.to_dict()
display(df_def_encar)

,ca_id,ca
ca_id,,
0,0,EL


In [57]:
df_def_pp_type = pd.DataFrame({'pt': ['GAS_LIN', 'SOL_PHO', 'HCO_ELC']})
df_def_pp_type.index.name = 'pt_id'
df_def_pp_type = df_def_pp_type.reset_index()
dict_pt = df_def_pp_type.set_index('pt').pt_id.to_dict()
display(df_def_pp_type)

,pt_id,pt
0,0,GAS_LIN
1,1,SOL_PHO
2,2,HCO_ELC


In [58]:
df_def_plant = pd.DataFrame({'pp': ['ND1_GAS_LIN', 'ND1_SOL_PHO', 'ND1_HCO_ELC'],
                             'pt_id': ['GAS_LIN', 'SOL_PHO', 'HCO_ELC'],
                             'nd_id': ['Node1'] * 3,
                             'fl_id': ['natural_gas', 'photovoltaics', 'hard_coal'],
                             'set_def_pr': [0, 1, 0],
                             'set_def_pp': [1, 0, 1],
                             'set_def_lin': [1, 0, 0],
                            })
df_def_plant.index.name = 'pp_id'
df_def_plant = df_def_plant.reset_index()

# translate columns to id using the previously defined def tables
df_def_plant = df_def_plant.assign(pt_id=df_def_plant.pt_id.replace(dict_pt),
                                   nd_id=df_def_plant.nd_id.replace(dict_nd),
                                   fl_id=df_def_plant.fl_id.replace(dict_fl))
dict_pp = df_def_plant.set_index('pp').pp_id.to_dict()

display(df_def_plant)

,pp_id,pp,pt_id,nd_id,fl_id,set_def_pr,set_def_pp,set_def_lin
0,0,ND1_GAS_LIN,0,0,0,0,1,1
1,1,ND1_SOL_PHO,1,0,2,1,0,0
2,2,ND1_HCO_ELC,2,0,1,0,1,0


In [59]:
df_def_profile = pd.DataFrame({'pf': ['SUPPLY_SOL_PHO', 'DMND_NODE1']})
df_def_profile.index.name = 'pf_id'
df_def_profile = df_def_profile.reset_index()
dict_pf = df_def_profile.set_index('pf').pf_id.to_dict()
df_def_profile


,pf_id,pf
0,0,SUPPLY_SOL_PHO
1,1,DMND_NODE1


In [60]:
eff_gas_min = 0.4
eff_gas_max = 0.6
cap_gas = 4000.
f0_gas = 1/eff_gas_min
f1_gas = 1/cap_gas * (f0_gas - 1/eff_gas_max)

df_plant_encar = pd.DataFrame({'pp_id': ['ND1_GAS_LIN', 'ND1_SOL_PHO', 'ND1_HCO_ELC'],
                               'ca_id': ['EL'] * 3,
                               'supply_pf_id': [None, 'SUPPLY_SOL_PHO', None],
                               'pp_eff': [None, None, 0.4],
                               'factor_lin_0': [f0_gas, None, None],
                               'factor_lin_1': [f1_gas, None, None],
                               'cap_pwr_leg': [2000, 1000, 4000],
                              })

df_plant_encar = df_plant_encar.assign(supply_pf_id=df_plant_encar.supply_pf_id.replace(dict_pf),
                                       pp_id=df_plant_encar.pp_id.replace(dict_pp),
                                       ca_id=df_plant_encar.ca_id.replace(dict_ca))
df_plant_encar



,pp_id,ca_id,supply_pf_id,pp_eff,factor_lin_0,factor_lin_1,cap_pwr_leg
0,0,0,None,NaN,2.5,0.000208,2000
1,1,0,0,NaN,NaN,NaN,1000
2,2,0,None,0.4,NaN,NaN,4000


In [61]:
df_node_encar = pd.DataFrame({'nd_id': ['Node1'], 'ca_id': ['EL'],
                              'dmnd_pf_id': ['DMND_NODE1']
                              })
df_node_encar = df_node_encar.assign(nd_id=df_node_encar.nd_id.replace(dict_nd),
                                     ca_id=df_node_encar.ca_id.replace(dict_ca),
                                     dmnd_pf_id=df_node_encar.dmnd_pf_id.replace(dict_pf))
df_node_encar

,nd_id,ca_id,dmnd_pf_id
0,0,0,1


In [62]:
df_fuel_node_encar = pd.DataFrame({'fl_id': ['natural_gas', 'hard_coal'],
                               'nd_id': ['Node1'] * 2,
                               'ca_id': ['EL'] * 2,
                               'vc_fl': [40, 10],
                              })
df_fuel_node_encar = df_fuel_node_encar.assign(fl_id=df_fuel_node_encar.fl_id.replace(dict_fl),
                                       nd_id=df_fuel_node_encar.nd_id.replace(dict_nd),
                                       ca_id=df_fuel_node_encar.ca_id.replace(dict_ca))
df_fuel_node_encar

,fl_id,nd_id,ca_id,vc_fl
0,0,0,0,40
1,1,0,0,10


In [63]:
hy = np.arange(24)
sp = (-(hy - 12)**2 + 50) / 60
sp[sp < 0] = 0

df_profsupply = pd.DataFrame({'supply_pf_id': [dict_pf['SUPPLY_SOL_PHO']] * len(hy),
                              'hy': hy, 'value': sp})

df_profsupply.head()

,supply_pf_id,hy,value
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0


In [64]:
hy = np.arange(24)
sp = np.sin(hy / 23 * np.pi *3) + 5

df_profdmnd = pd.DataFrame({'dmnd_pf_id': [dict_pf['DMND_NODE1']] * len(hy),
                              'hy': hy, 'value': sp})

df_profdmnd.head()

,dmnd_pf_id,hy,value
0,1,0,5.000000
1,1,1,5.398401
2,1,2,5.730836
3,1,3,5.942261
4,1,4,5.997669


In [66]:
for dftb, tbname in [(df_def_node, 'def_node'),
                   (df_def_plant, 'def_plant'),
                   (df_def_fuel, 'def_fuel'),
                   (df_def_encar, 'def_encar'),
                   (df_node_encar, 'node_encar'),
                   (df_def_pp_type, 'def_pp_type'),
                   (df_plant_encar, 'plant_encar'),
                   (df_def_profile, 'def_profile'),
                   (df_fuel_node_encar, 'fuel_node_encar'),
                   (df_profsupply, 'profsupply'),
                   (df_profdmnd, 'profdmnd')]:
    dftb.to_csv('introductory_example_files/{}.csv'.format(tbname), index=False)
    

In [14]:
import os

mkwargs = {}
iokwargs = {'data_path': os.path.abspath('introductory_example_files/'),
            'output_target': 'hdf5',
            'cl_out': 'introductory_example_files/output.hdf5'}
nsteps = [('swco', 3, np.linspace),  # CO2 emission price
         ]
ml = model_loop.ModelLoop(nsteps=nsteps, mkwargs=mkwargs, iokwargs=iokwargs)
print(ml.io.datrd.data_path)

> 21:22:55 - ERROR - grimsel.core.io - 'No object named def_run in the file'
> 21:22:55 - WARNING - grimsel.core.io - reset_hdf_file: Could not determine max_run_id ... setting to None.



~~~~~~~~~~~~~~~   WARNING:  ~~~~~~~~~~~~~~~~
You are about to delete existing file introductory_example_files/output.hdf5.
The maximum run_id is None.

Hit enter to proceed.

/mnt/data/Dropbox/GRIMSEL_SOURCE/grimsel/notebooks/introductory_example_files


In [15]:
IO._close_all_hdf_connections()
ml.init_run_table()
ml.df_def_run

print(ml.io.datrd.data_path)


/mnt/data/Dropbox/GRIMSEL_SOURCE/grimsel/notebooks/introductory_example_files


Closing remaining open files:introductory_example_files/output.hdf5...done


In [16]:
logger.setLevel(1000)
ml.io.read_model_data()


In [17]:
ml.m.init_maps()

ml.m.map_to_time_res()


# %
ml.io.write_runtime_tables()

ml.m.get_setlst()
ml.m.define_sets()
ml.m.add_parameters()
ml.m.define_variables()
ml.m.add_all_constraints()
ml.m.init_solver()
ml.io.init_output_tables()
ml.select_run(0)


AttributeError: 'NoneType' object has no attribute 'empty'